In [2]:
#install MlFlow

#!pip install mlflow

In [3]:
#Check whether MLFlow is installed or not

#!mlflow

In [4]:
#!python -m pip install --upgrade pip

In [5]:
#let's do practically
import mlflow
import mlflow.sklearn

mlflow.set_experiment('LearnML-Demo')

2022/07/26 13:22:39 INFO mlflow.tracking.fluent: Experiment with name 'LearnML-Demo' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/Shiwani/Desktop/mlruns/1', experiment_id='1', lifecycle_stage='active', name='LearnML-Demo', tags={}>

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn
import sys
import os

In [7]:
def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

In [9]:
data = pd.read_csv("winequality-red.csv",delimiter=";")

In [10]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [11]:
# Read the wine-quality csv file


def train_model(alpha,l1_ratio):
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [12]:
train_model(0.5,0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7400151509935501
  MAE: 0.5818334779051892
  R2: 0.1419854503863035


In [13]:
train_model(0.2,0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7188506044179432
  MAE: 0.5613261641949034
  R2: 0.24130642861247642


In [14]:
train_model(0.1,0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.6709449389269222
  MAE: 0.533036497991889
  R2: 0.3131414234488077


In [15]:
train_model(0.4,0.1)

Elasticnet model (alpha=0.400000, l1_ratio=0.100000):
  RMSE: 0.7174110914410095
  MAE: 0.5675778918647753
  R2: 0.2142235340068811


In [16]:
train_model(0.1,0.4)

Elasticnet model (alpha=0.100000, l1_ratio=0.400000):
  RMSE: 0.6675442103076676
  MAE: 0.5228933556358347
  R2: 0.23033762646869427


In [ ]:
!mlflow ui